In [12]:
import os
import numpy as np
import argparse


import pandas as pd
from sklearn.decomposition import PCA
from operator import itemgetter
import random
import matplotlib.pyplot as plt
import umap.umap_ as umap
import time

import torch
from spiral.main import SPIRAL_integration
from spiral.layers import *
from spiral.utils import *
from spiral.CoordAlignment import CoordAlignment
R_dirs="/home/tguo/tguo2/miniconda3/envs/stnet/lib/R"
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

SPIRAL integration

In [4]:
############train#############
torch.cuda.cudnn_enabled = False
seed=0
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
random.seed(seed)

read in orignal data (feat:ncell*ngene; edge:nedge*2; meta:ncell*2)

In [5]:
dirs="/data02/tguo/space_batch_effect/mouse_brain/"
samples=["all_posterior1","all_posterior2","all_anterior1","all_anterior2"]
# samples=["pa_anterior1","pa_posterior1"]
SEP=','
net_cate='_KNN_'
knn=6

N_WALKS=knn
WALK_LEN=1
N_WALK_LEN=knn
NUM_NEG=6


feat_file=[]
edge_file=[]
meta_file=[]
flags=''
for i in range(len(samples)):
    feat_file.append(dirs+"gtt_input/"+str(samples[i])+"_mat.csv")
    edge_file.append(dirs+"gtt_input/"+str(samples[i])+"_edge"+net_cate+str(knn)+".csv")
    meta_file.append(dirs+"gtt_input/"+str(samples[i])+"_meta.csv")
    flags=flags+'_'+samples[i]
N=pd.read_csv(feat_file[0],header=0,index_col=0).shape[1]
M=4

In [6]:
parser = argparse.ArgumentParser()

parser.add_argument('--AEdims', type=list, default=[N,[512],32], help='Dim of encoder.')
parser.add_argument('--AEdimsR', type=list, default=[32,[512],N], help='Dim of decoder.')
parser.add_argument('--GSdims', type=list, default=[512,32], help='Dim of GraphSAGE.')
parser.add_argument('--zdim', type=int, default=32, help='Dim of embedding.')
parser.add_argument('--znoise_dim', type=int, default=4, help='Dim of noise embedding.')
parser.add_argument('--CLdims', type=list, default=[4,[],M], help='Dim of classifier.')
parser.add_argument('--DIdims', type=list, default=[28,[32,16],M], help='Dim of discriminator.')
parser.add_argument('--beta', type=float, default=1.0, help='weight of GraphSAGE.')
parser.add_argument('--agg_class', type=str, default=MeanAggregator, help='Function of aggregator.')
parser.add_argument('--num_samples', type=str, default=20, help='number of neighbors to sample.')

parser.add_argument('--N_WALKS', type=str, default=6, help='number of walks of random work for postive pairs.')
parser.add_argument('--WALK_LEN', type=str, default=1, help='walk length of random work for postive pairs.')
parser.add_argument('--N_WALK_LEN', type=str, default=6, help='number of walks of random work for negative pairs.')
parser.add_argument('--NUM_NEG', type=str, default=6, help='number of negative pairs.')


parser.add_argument('--epochs', type=int, default=50, help='Number of epochs to train.')
parser.add_argument('--batch_size', type=int, default=64, help='Size of batches to train.')
parser.add_argument('--lr', type=float, default=1e-3, help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4, help='Weight decay.')
parser.add_argument('--alpha1', type=float, default=N, help='Weight of decoder loss.')
parser.add_argument('--alpha2', type=float, default=1, help='Weight of GraphSAGE loss.')
parser.add_argument('--alpha3', type=float, default=1, help='Weight of classifier loss.')
parser.add_argument('--alpha4', type=float, default=1, help='Weight of discriminator loss.')
parser.add_argument('--lamda', type=float, default=1, help='Weight of GRL.')
parser.add_argument('--Q', type=float, default=10, help='Weight negative loss for sage losss .')

params,unknown=parser.parse_known_args()

In [7]:
SPII=SPIRAL_integration(params,feat_file,edge_file,meta_file)

In [8]:
SPII.train()

--------------------------------
Training.
processed: 3:   6%|▌         | 3/50 [15:58<4:10:21, 319.60s/it, CLAS_loss=1.36, DISC_loss=1.19, GS_loss=1.11, total_loss=609]


KeyboardInterrupt: 

SPIRAL alignment

In [10]:
#######step1: louvain clustering (seurat package) using spiral embeddings to obtain cluster labels############
#######step2: coordination alignment########
dirs="/data02/tguo/space_batch_effect/simulate/"
batch="_1_2"
types="_3batch_types5"
flags=batch+types
clust_cate='louvain'
meta_file=[]
coord_file=[]
samples=['1','2','3']
for i in np.arange(len(samples)):
    meta_file.append(dirs+"gtt_input/meta"+samples[i]+flags+".csv")
    coord_file.append(dirs+"gtt_input/coord"+samples[i]+flags+".csv")
embed_file=dirs+"gtt_output/AGC_embed"+flags+"_xent.csv"
cluster_file=dirs+'metrics/'+clust_cate+'_clust'+flags+'.csv'
input_file=[meta_file,coord_file,embed_file,cluster_file]
output_dirs=dirs+"gtt_output/coordinate_file/"
ub=['Batch1','Batch2','Batch3']

In [11]:
alpha=0.5
types="weighted_mean"
CA=CoordAlignment(input_file=input_file,output_dirs=output_dirs,ub=ub,flags=flags,clust_cate=clust_cate,R_dirs=R_dirs,alpha=alpha,types=types)
New_Coord=CA.New_Coord

/home/tguo/tguo2/miniconda3/envs/stnet/lib/python3.6/site-packages/pandas/core/indexing.py:1758: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)
/home/tguo/tguo2/miniconda3/envs/stnet/lib/python3.6/site-packages/pandas/core/indexing.py:1758: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)
/home/tguo/tguo2/miniconda3/envs/stnet/lib/python3.6/site-packages/pandas/core/indexing.py:1758: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u